# Why 1.5°C Matters - A View from Canada

The world is spiraling out of control. We've passed the critical 1.5°C threshold, and it's not just a number—it's the tipping point beyond which the devastating effects of climate change start to snowball. **This notebook is my attempt to channel that frustration into something constructive: understanding.**

We’re going to dive into why limiting global warming to 1.5°C isn’t just important—it’s *essential* for our survival. Whether you’re someone who struggles with graphs or a scientist familiar with climate models, the data will speak for itself. For those unfamiliar with Jupyter Notebooks, don’t worry! The format allows us to combine text with executable code. Feel free to skip the code, but if you’re interested, it’s there for you to explore.

The Paris Agreement set the goal of limiting warming to 1.5°C. But why is that number so crucial? I’m going to show, using official climate data (with links), why this is one of the most critical numbers in human history.

## What Does 1.5°C Really Mean?

You’ve probably heard about the impacts of a 1°C, 1.5°C, or even 2°C change. But what does that really mean in terms of the planet? A 1°C shift might sound small, but on a global scale, it’s monumental. Let’s break this down and explore the true impact.

## How Much Have We Already Warmed?

To put this in perspective, we’ll look at data from four cities across Canada:

- **Sault Ste. Marie** (Northern Ontario, my hometown)
- **Toronto** (Southern Ontario, warmer than Sault Ste. Marie)
- **Whitehorse** (Capital of the Yukon)
- **Yellowknife** (Capital of the Northwest Territories)

We’ll use data from [WeatherStats](https://www.weatherstats.ca) and global temperature data from the [University of Maine's Climate Reanalyzer](https://climatereanalyzer.org/).

Our goal is to visualize:

- **Dates** on one axis
- **Rolling averages of annual temperatures** on the other

By using rolling averages, we’ll cut through the noise of seasonal fluctuations and focus on the long-term climate trends. Let’s graph the data and take a hard look at how much change has already occurred. The x-axis will represent the dates, and the y-axis will show the rolling average of temperatures over the past year.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

def process_hourly_data(hourly_file):
    """
    Process hourly temperature data to calculate daily averages and rolling averages.

    Args:
        hourly_file (str): filename with hourly temperature data in CSV format.

    Returns:
        DataFrame: Processed daily data with 'date' and 'rolling_average_temp' columns.
    """
    # Read the hourly data
    hourly_data = pd.read_csv(hourly_file, low_memory=False)
    
    # Convert 'unixtime' to 'date' and calculate daily average temperature
    hourly_data['date'] = pd.to_datetime(hourly_data['unixtime'], unit='s').dt.date
    daily_data = hourly_data.groupby('date')['temperature'].mean().reset_index()
    
    return hourly_data, daily_data

def process_rolling_daily(daily_data):
    """
    Calculate the rolling average temperature over a 365-day window.

    Args:
        daily_data (DataFrame): DataFrame with 'date' and 'temperature' columns.

    Returns:
        DataFrame: DataFrame with 'date' and 'rolling_average_temp' columns.
    """
    result = daily_data.copy()
    result['rolling_average_temp'] = daily_data['temperature'].rolling(window=365).mean()
    result = result.dropna(subset=['rolling_average_temp'])
    
    return result

def process_global_temperature_json(json_file_path):
    """
    Process global temperature data from a JSON file to calculate daily averages and rolling averages.

    Args:
        json_file_path (str): Path to the JSON file containing global temperature data.

    Returns:
        DataFrame: Processed global temperature data with 'date' and 'rolling_average_temp' columns.
    """
    # Load global temperature data from JSON
    with open(json_file_path, 'r') as file:
        global_temperature_data = json.load(file)
    
    # Transform the JSON into a DataFrame
    rows = []
    for year_data in global_temperature_data:
        year = year_data["name"]
        daily_temps = year_data["data"]
        for day, temp in enumerate(daily_temps, start=1):
            rows.append({"Year": year, "Day": day, "Temperature": temp})
    
    global_temperature_df = pd.DataFrame(rows)
    
    # Clean the data (filter nulls, handle non-leap years)
    global_temperature_df = global_temperature_df[global_temperature_df['Temperature'].notnull()]
    global_temperature_df = global_temperature_df[~global_temperature_df['Year'].str.contains('-')]
    
    # Convert 'Year' and 'Day' to 'date' and calculate rolling average
    global_temperature_df['date'] = (pd.to_datetime(global_temperature_df['Year'], format='%Y') + 
                                      pd.to_timedelta(global_temperature_df['Day'] - 1, unit='D')).dt.date
    global_daily = global_temperature_df[['date', 'Temperature']].rename(columns={'Temperature': 'average_daily_temp'})
    
    global_daily['rolling_average_temp'] = global_daily['average_daily_temp'].rolling(window=365, min_periods=365).mean()
    global_daily = global_daily.dropna(subset=['rolling_average_temp'])
    global_daily = global_daily[['date', 'rolling_average_temp']]
    
    return global_daily

def plot_rolling_averages(dataframe_dict):
    """
    Plots the 365-day rolling average temperatures for multiple cities and global average.
    
    Args:
        dataframe_dict (dict): A dictionary where the keys are city names (strings) 
                               and the values are DataFrames containing 'date' and 'rolling_average_temp'.
    """
    plt.figure(figsize=(12, 6))
    plt.title("365-Day Rolling Average Temperatures", fontsize=16)
    
    # Loop through each city and plot the data
    for city, data in dataframe_dict.items():
        color = color_mapping.get(city, "#000000")  # Default to black if city not in color_mapping
        plt.plot(data['date'], data['rolling_average_temp'], label=city, color=color, alpha=0.7)
    
    # Add labels and grid
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("365-Day Rolling Average Temperature (°C)", fontsize=14)
    plt.xticks(rotation=45)  # Rotate date labels for better readability
    plt.legend()
    plt.grid(True)
    plt.tight_layout()  # Ensures everything fits without overlapping
    plt.show()

# Deterministic color mapping for each city and global average
color_mapping = {
    "Sault Ste. Marie": "#1f77b4",  # Blue
    "Toronto": "#ff7f0e",          # Orange
    "Whitehorse": "#2ca02c",       # Green
    "Yellowknife": "#d62728",      # Red
    "Global Average": "#9467bd"    # Purple
}

# Example usage for cities:
city_files = {
    'Sault Ste. Marie': '/kaggle/input/climate-data/weatherstats_saultstemarie_hourly.csv',
    'Toronto': '/kaggle/input/climate-data/weatherstats_toronto_hourly.csv',
    'Whitehorse': '/kaggle/input/climate-data/weatherstats_whitehorse_hourly.csv',
    'Yellowknife': '/kaggle/input/climate-data/weatherstats_yellowknife_hourly.csv'
}

city_data = {}
city_hourly_data = {}
for city, file in city_files.items():
    hourly_data, daily_data = process_hourly_data(file)
    city_hourly_data[city] = hourly_data
    city_data[city] = process_rolling_daily(daily_data)

# Example usage for global temperature data:
city_data['Global Average'] = process_global_temperature_json('/kaggle/input/global-average-air-temperature/era5_world_t2_day.json')

# Call the plotting function
plot_rolling_averages(city_data)

## Initial Data Visualization

Now that we’ve seen the data, here are some critical takeaways from the initial graph:

- Our cities show far more **fluctuation** than global averages, revealing the local intensity of climate change’s impact. Our temperature swings are getting bigger.
- Our cities are warming **faster** than the global average, showing us how much faster land warms than the ocean. The global average includes a lot of ocean temperatures, as the Earth is mostly covered by oceans.

## Cleaning and Refining the Data

The data spans different time periods across cities, and in order to make meaningful comparisons, we need consistency. Some cities have more data than others, and we must focus on the years where all cities' data overlap. To achieve this, we’ll trim the datasets to the years where we have complete information across all locations.

Next, we’ll calculate the **average temperatures** for each city during the **first 20 years** of the trimmed dataset (from **July 31, 1962**, to **July 30, 1982**). This baseline provides us with a solid reference point to measure the alarming rate of change since then. While you can choose a different baseline if you'd prefer, this is the one I've chosen for this analysis.

With the data cleaned and aligned, we can now dive into the magnitude of these changes. How much difference does a single degree make, and what does it mean for the world?

In [ ]:
# Optimizing date range overlap calculation
dates = [df['date'] for df in city_data.values()]
latest_start_date = max([df.min() for df in dates])
earliest_end_date = min([df.max() for df in dates])

# Filter data to match the date range where all datasets overlap
city_data = {city: df[(df['date'] >= latest_start_date) & (df['date'] <= earliest_end_date)] for city, df in city_data.items()}

# Plot rolling averages for all cities and global average
plot_rolling_averages(city_data)

# Define 20 years in days (including leap years)
twenty_years = 365 * 20 + 5

# Calculate the 20-year baseline average temperature for each city
temperature_changes = {city: df['rolling_average_temp'][:twenty_years].mean() for city, df in city_data.items()}

# Get the most recent temperature for each city
today_temperatures = {city: df['rolling_average_temp'].iloc[-1] for city, df in city_data.items()}

# Calculate the temperature change since the 1962-1982 baseline
temperature_deltas = {city: today - baseline for city, (baseline, today) in zip(temperature_changes.keys(), zip(temperature_changes.values(), today_temperatures.values()))}

# Data for histogram plot
locations = list(temperature_deltas.keys())
changes = list(temperature_deltas.values())
colors = [color_mapping[location] for location in locations]

# Plot the temperature change since 1962-1982
plt.bar(locations, changes, color=colors, alpha=0.7)
plt.xlabel("Locations")
plt.ylabel("Temperature Change (°C)")
plt.title("Temperature Change Since 1962-1982")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate average temperatures for baseline (1962-1982) and today
baseline_temps = {city: df['rolling_average_temp'][:twenty_years].mean() for city, df in city_data.items()}
today_temps = {city: df['rolling_average_temp'].iloc[-1] for city, df in city_data.items()}

# Prepare data for the histogram plot
locations = list(city_data.keys())
baseline_values = [baseline_temps[city] for city in locations]
today_values = [today_temps[city] for city in locations]

# Create space between bars for different locations by shifting the x positions slightly
bar_width = 0.35  # Width of the bars
x_positions_baseline = range(len(locations))  # Positions for baseline bars
x_positions_today = [x + bar_width for x in x_positions_baseline]  # Shift today bars to the right

# Plotting the histogram with bars for both baseline and today
plt.figure(figsize=(10, 6))
plt.bar(x_positions_baseline, baseline_values, width=bar_width, label='1962-1982 (Baseline)', color='lightblue', alpha=0.7)
plt.bar(x_positions_today, today_values, width=bar_width, label='Today', color='salmon', alpha=0.7)

# Add labels and title
plt.xlabel("Locations", fontsize=12)
plt.ylabel("Average Temperature (°C)", fontsize=12)
plt.title("Average Temperatures: 1962-1982 vs Today", fontsize=14)

# Add xticks with location names
plt.xticks([x + bar_width / 2 for x in x_positions_baseline], locations, rotation=45)

# Add a legend
plt.legend()

# Adjust layout to avoid overlap
plt.tight_layout()

# Show the plot
plt.show()

We can already see that temperatures are rising everywhere—and all cities are warming faster than the global average, with some cities experiencing far more dramatic changes than others. However, when we compare temperature differences between cities, it’s staggering how just a few degrees can profoundly alter the climate. For example, while Sault Ste. Marie and Toronto are both located along the Great Lakes and experience similar temperature fluctuations, Sault Ste. Marie lies about 300 km farther north. This results in a temperature difference of just 3°C, but that small shift has a massive impact on what crops can thrive—apples and plums in Sault Ste. Marie, compared to peaches and grapes in Toronto. It’s a powerful reminder that even seemingly small temperature changes can have profound effects on local ecosystems, agriculture, and, ultimately, the food we rely on.

Let’s now dive deeper into these changes. We’ll examine shifts in growing zones and enhance our rolling average graph with both the baseline (1962-1982) and today’s temperatures for a clearer visual comparison. Additionally, since Whitehorse has now surpassed the 0°C mark, we’ll label permafrost and non-permafrost zones on the graph to make this shift even more striking and easier to grasp.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define USDA Hardiness Zones (zone boundaries in °C for minimum temperature)
usda_zones = {
    '0a': -53.9, '0b': -51.1, '1a': -48.3, '1b': -45.6, '2a': -42.8, '2b': -40, '3a': -37.2, '3b': -34.4,
    '4a': -31.7, '4b': -28.9, '5a': -26.1, '5b': -23.3, '6a': -20.6, '6b': -17.8, '7a': -15, '7b': -12.2,
    '8a': -9.4, '8b': -6.7, '9a': -3.9, '9b': -1.1, '10a': 1.7, '10b': 4.4, '11a': 7.2, '11b': 10,
    '12a': 12.8, '12b': 15.6, '13a': 18.3, '13b': 99
}

# Function to determine USDA zone based on coldest temperature
def get_usda_zone(coldest_temp):
    for zone, temp in usda_zones.items():
        if coldest_temp <= temp:
            return zone
    return '13b'  # If colder than zone 13b, assume zone 13b

# Function to find the coldest winter temperature for each year (July 1 to June 30)
def find_coldest_winter_temperatures(df, start_month=7, end_month=6):
    # Filter data to match the winter period: July 1 to June 30 of the following year
    df['date'] = pd.to_datetime(df['date'])
    
    # Initialize a list to store coldest temperatures for each winter period (year)
    coldest_temperatures = []

    # Loop over each year (we'll extract all data starting July 1 and ending June 30)
    for year in df['date'].dt.year.unique():
        winter_data = df[((df['date'].dt.year == year) & 
                          ((df['date'].dt.month >= start_month) | (df['date'].dt.month <= end_month)))]
        # Find the coldest temperature during this winter period
        coldest_temp = winter_data['temperature'].min()
        coldest_temperatures.append({'year': year, 'coldest_temp': coldest_temp})
    
    return coldest_temperatures

# Dataframe to store results
results = []

# Process data for each city
for city, df in city_hourly_data.items():
    coldest_temperatures = find_coldest_winter_temperatures(df)
    for entry in coldest_temperatures:
        usda_zone = get_usda_zone(entry['coldest_temp'])
        results.append({'city': city, 'year': entry['year'], 'coldest_temp': entry['coldest_temp'], 'usda_zone': usda_zone})

# Convert results to DataFrame
usda_comparison_df = pd.DataFrame(results)

# Create a dictionary to convert USDA zones to numerical scale for plotting
zone_to_num = {zone: i for i, zone in enumerate(usda_zones.keys())}

# Convert USDA zones to a numerical scale for plotting
usda_comparison_df['zone_num'] = usda_comparison_df['usda_zone'].map(zone_to_num)

# Plotting the data: Line graph with USDA zone number for each year
plt.figure(figsize=(12, 6))

# For each city, plot the USDA zone number over time
for city, df in city_data.items():
    city_data_for_plot = usda_comparison_df[usda_comparison_df['city'] == city]
    plt.plot(city_data_for_plot['year'], city_data_for_plot['zone_num'], marker='o', label=f'{city}')

# Labels and title
plt.xlabel("Year", fontsize=12)
plt.ylabel("USDA Zone", fontsize=12)
plt.title("USDA Hardiness Zone by Year for Each City", fontsize=14)
plt.xticks(rotation=45)

# Map the numerical y-axis values back to the USDA zones
plt.yticks(ticks=range(len(usda_zones)), labels=list(usda_zones.keys()))

# Add legend
plt.legend()
# Adjust layout to make everything fit nicely
plt.tight_layout()
# Show the plot
plt.show()

## Fluctuations in Hardiness Zones: A Climate Crisis

The fluctuations in hardiness zones—how we classify what can grow where—are becoming increasingly erratic, particularly in winter. This means that plant species are spreading into new regions at an unprecedented pace. The idea of managing invasive species is no longer practical. That ship has sailed.

Our ecosystems, already under strain, are set to collapse. These changes are too rapid and too large to adapt to. And they are accelerating. For example, we've seen entire forest ecosystems in the northern and alpine areas of the world collapse. The result? Massive wildfires that are now a regular occurrence.

The question becomes: will anything survive? At this point, we can only hope some invasive species will be able to recolonize the land, or that some hardier species can hold their ground. But the real concern is that, without intervention, large areas could degrade into desert-like conditions, a process known as desertification.

In [ ]:
# Line styles
line_styles = {
    "1962-1982": "dashed",
    "Today": "dotted"
}

# Dates for the horizontal lines
start_date = city_data['Sault Ste. Marie']['date'].min()
end_date = city_data['Sault Ste. Marie']['date'].max()

# Plot the rolling averages for all cities plus the global temperatures
plt.figure(figsize=(12, 6))
plt.title("365-Day Rolling Average Temperatures")

# Define temperature thresholds (y-axis based)
plt.axhspan(ymin=0, ymax=17, facecolor='pink', alpha=0.7, label='Not Permafrost')
plt.axhspan(ymin=-10, ymax=0, facecolor='lightblue', alpha=0.7, label='Permafrost')

# Plot rolling averages dynamically for each city
for city in ['Sault Ste. Marie', 'Toronto', 'Whitehorse', 'Yellowknife', 'Global Average']:
    plt.plot(city_data[city]['date'], city_data[city]['rolling_average_temp'], label=city, alpha=0.7, color=color_mapping[city])

# Add horizontal lines for 1962-1982 averages and today's averages
for city, avg in baseline_temps.items():
    plt.hlines(avg, start_date, end_date, colors=color_mapping[city], linestyles=line_styles["1962-1982"], label=f"{city} 1962-1982 Avg")
    plt.hlines(today_temps[city], start_date, end_date, colors=color_mapping[city], linestyles=line_styles['Today'], label=f"{city} Today")

plt.xlabel("Date")
plt.ylabel("365-Day Rolling Average Temperature (°C)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left') 
plt.grid(True)
plt.show()

Our graph is a lot busier now, but it also carries a lot more critical information.

Toronto shows the largest increase, but it’s the other cities that are more alarming. Sault Ste. Marie ended the year warmer than Toronto during the baseline period, which is troubling. Yellowknife has warmed to nearly the same level as Whitehorse, but the most concerning shift is happening in Whitehorse. It’s crossing a critical threshold—its average temperature is no longer below freezing.

### The Melting of Permafrost

When temperatures remain below zero on average, the ground stays frozen at certain depths, never fully thawing. This is how permafrost forms—it freezes deeper than it melts. However, when temperatures consistently rise above zero, the frozen ground will eventually thaw. It won’t happen all at once, like glaciers, because permafrost is thick. But, like glaciers, if the temperature stays warm enough, it will melt over time.

**The permafrost has started to melt.**

This is catastrophic news. It’s *apocalyptically bad news*. Let’s take a deeper look at why this is such a dire situation.

For this, we’ll keep things relatively straightforward. We'll generate a basic histogram showing the CO2 levels in the atmosphere: when records began, today’s levels, how much is stored in the permafrost, and how much we’ll have if all of it melts. 

To get the first two, I’ll retrieve the CO2 data from the Mauna Loa Observatory (available at [co2.earth](https://www.co2.earth/22-co2-now/121-mauna-loa-co2)) and convert it into tons. For the third data point, I’ll look up the amount of CO2 stored in the permafrost—it’s not a time series, so I’ll just gather that value. Since CO2 levels, like temperatures, follow a seasonal cycle, I’ll calculate rolling averages to smooth out the noise.

Let’s load the data and check it out. At this stage, it will be in parts per million (ppm).

In [ ]:
# a bunch of comments at the beginning of the file
daily_co2 = pd.read_csv('/kaggle/input/atmospheric-co2-levels/co2_daily_mlo.csv', header=None, names=['year', 'month', 'day', 'year_as_float', 'co2_ppm'], skiprows=32)
daily_co2['rolling_average_ppm'] = daily_co2['co2_ppm'].rolling(window=365).mean()
daily_co2 = daily_co2.dropna(subset=['rolling_average_ppm'])

# Plot the atmospheric CO2 levels by date
plt.figure(figsize=(12, 6))
plt.title("Atmospheric CO2 levels (ppm)")
plt.plot(daily_co2['year_as_float'], daily_co2['rolling_average_ppm'], label='CO2 (ppm)', alpha=0.7)
plt.ylim(0, daily_co2['rolling_average_ppm'].max() * 1.1)  # Ensure y-axis starts at 0 and allows some headroom
plt.xlabel("Year")
plt.ylabel("Carbon Dioxide Concentration in Parts per Million")
plt.legend()
plt.grid(True)
plt.show()

Alright, this is what we expect to see: CO2 levels have risen from around 330 parts per million (ppm) to over 420 ppm in the last 50 years. Now, let's convert this into tons.

Next, we’ll compare these numbers to the carbon stored in the permafrost, which holds around 1,460 to 1,600 gigatons of organic carbon. When this carbon decomposes, it turns into either carbon dioxide (CO2) or methane (CH4), depending on the conditions—aerobic or anaerobic, respectively. If it all decomposed aerobically, it would release about 5,360 to 5,870 gigatons of CO2.

Given that methane is roughly 85 times more potent than CO2 as a greenhouse gas over a 20-year period, we can hope it decomposes into CO2 rather than methane. Unfortunately, it looks like that’s exactly what will happen.

It’s important to note that the ocean plays a role in this process. It absorbs about 25-30% of excess CO2, with the remaining CO2 staying in the atmosphere.

For the sake of this scenario, let’s use the lowest-end estimates—5,360 gigatons of CO2—which assumes the best-case situation: all the permafrost melts and decomposes as carbon dioxide, with no methane produced, and the ocean absorbs 30% of the CO2.

This is the absolute best-case scenario if the permafrost melts and the carbon is released into the atmosphere. Even in this scenario, the implications are catastrophic.

In [ ]:
def ppm_to_tons_co2(ppm):
    """
    Convert atmospheric CO2 levels in ppm to tons of CO2.

    Args:
        ppm (float): CO2 concentration in parts per million.

    Returns:
        float: Mass of CO2 in tons.
    """
    # Constants
    mass_of_atmosphere_tons = 5.5 * 1e15  # Total mass of Earth's atmosphere in tons (5.5 quadrillion tons)
    molar_mass_co2 = 44.01  # Molar mass of CO2 in g/mol
    molar_mass_air = 28.97  # Molar mass of average air in g/mol

    # Conversion factor
    fractional_mass = molar_mass_co2 / molar_mass_air * 1e-6

    # Calculate total CO2 mass in tons
    co2_mass_tons = ppm * mass_of_atmosphere_tons * fractional_mass
    return co2_mass_tons

# Calculate CO2 mass at the start, today, and from permafrost
starting_co2_in_tons = ppm_to_tons_co2(daily_co2['rolling_average_ppm'].iloc[0])
todays_co2_in_tons = ppm_to_tons_co2(daily_co2['rolling_average_ppm'].iloc[-1])
low_end_permafrost_co2_in_tons = 5360 * 0.7 * 1e9  # 5360 gigatons CO2, absorbing 30% by ocean
total_expected_co2_in_tons = todays_co2_in_tons + low_end_permafrost_co2_in_tons

# Create a histogram to compare CO2 levels
plt.figure(figsize=(10, 6))
plt.bar(['Starting CO2', 'Current CO2', 'Low-End Permafrost CO2', 'After Permafrost Melts'], 
        [starting_co2_in_tons / 1e9, todays_co2_in_tons / 1e9, low_end_permafrost_co2_in_tons / 1e9, total_expected_co2_in_tons / 1e9], 
        color=['blue', 'red', 'green', 'black'], alpha=0.8)
plt.title('Comparison of CO2 Levels in Gigatons')
plt.xlabel('CO2 Categories')
plt.ylabel('CO2 Levels (Gigatons)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

The world is falling apart because we’ve gone from the blue bar to the red bar over the past 50 years. The green bar will soon be added as the permafrost thaws—releasing even more carbon into the atmosphere. Much of it will come out as methane, which is **85 times** more potent as a greenhouse gas in the short term, before eventually decomposing into CO2.

And at the rate things are going, it won’t take another 50 years for this to happen. Remember, this scenario assumes we go carbon negative **today**—we’re already far beyond the point where "net zero" will be enough. The permafrost isn’t the only climate tipping point we’re facing. Losing even a fraction—just a quarter—of the permafrost would release more carbon into the atmosphere than humanity has contributed over the past 50 years. And once again, that’s the best-case scenario, where it all decomposes into CO2.

Without **atmospheric CO2 removal** (and ideally methane removal as well), and without going **heavily carbon negative** starting **immediately**, our species may not survive until the end of the century much less limit warming to 2°C at this point by the end of the century. The only way to avoid a runaway greenhouse effect that could wipe out life as we know it is if we start removing carbon at the scale of tens or hundreds of gigatons per year.

So… get started. Or don’t.

**Happy Climate Change.**

Buckle up, thank a Boomer, and good luck.